In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def f(x):
    return 3*x**2 - 4*x + 5

In [ ]:
f(3.0)

In [ ]:
X = np.arange(-5, 5, 0.1)
Y = f(X)
plt.plot(X, Y)

In [ ]:
h = 0.000001
x = 2/3
derivative = (f(x + h) - f(x - h)) / (2 *h)

# ~0.000
derivative

In [ ]:
# lets get more complex
a = 2.0
b = -3.0
c = 10.0
d = a*b + c

# 4.0
print(d)

In [ ]:
h = 0.0001

a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
a += h
d2 = a*b + c

derivative_a = (d2 - d1) / h
print(f"d1: {d1}")  # 4.0
print(f"d2: {d2}")  # 3.996..
print(f"derivative_a: {derivative_a}")  # -3.000..

In [ ]:
h = 0.0001

a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
b += h
d2 = a*b + c

derivative_b = (d2 - d1) / h
print(f"d1: {d1}")  # 4.0
print(f"d2: {d2}")  # 4.0002
print(f"derivative_b: {derivative_b}")   # 2.0000..

In [ ]:
h = 0.0001

a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
c += h
d2 = a*b + c

derivative_c = (d2 - d1) / h
print(f"d1: {d1}")  # 4.0
print(f"d2: {d2}")  # 4.0001
print(f"derivative_c: {derivative_c}")  # 0.999999..

In [ ]:
class Value:
    def __init__(self, data, label="", _in=(), _op=""):
        self.data = data
        self.label = label
        self._inputs = set(_in)
        self._operation = _op
        self._grad = 0.0
    
    def __add__(self, other):
        return Value(self.data + other.data, _in=(self, other), _op="+")

    def __sub__(self, other):
        return Value(self.data - other.data, _in=(self, other), _op="-")

    def __mul__(self, other):
        return Value(self.data * other.data, _in=(self, other), _op="*")
    
    def __div__(self, other):
        return Value(self.data / other.data, _in=(self, other), _op="/")
    
    def __repr__(self):
        return f"Value({self.data})"
    

In [ ]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2, label='f')
L = d * f; L.label='L'

# Value(-8.0)
print(L)

In [ ]:
from graphviz import Digraph

def build_dot_graph(root_node):
    def add_inputs(parent_node, all_nodes):
        if parent_node not in all_nodes:
            all_nodes.add(parent_node)
            for input_node in parent_node._inputs:
                add_inputs(input_node, all_nodes)

    all_nodes = set()
    add_inputs(root_node, all_nodes)

    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
    for node in all_nodes:
        node_label=f'{{ {node.label} | {{ d={node.data} | g={node._grad} }}}}'

        dot.node(str(id(node)), node_label, shape="record")

        if node._inputs:
            op_node_id = str(id(node))+'_'+node._operation
            dot.node( op_node_id, label=node._operation )
            dot.edge(op_node_id, str(id(node)))
            for input_node in node._inputs:
                dot.edge(str(id(input_node)), op_node_id)
    return dot

In [ ]:
build_dot_graph(L)

In [ ]:
# dL/dL = 1.0
L._grad = 1.0
# dL/df = 4.0  # value of d in d*f
f._grad = 4.0
# dL/dd = -2.0  # value of f in d*f
d._grad = -2.0

# dd/dc = 1.0  # addition
# dL/dc = dd/dc * dL/dd = 1.0 * -2.0 = -2.0
c._grad = -2.0

# dd/de = 1.0  # addition
# dL/de = dd/de * dL/dd = 1.0 * -2.0 = -2.0
e._grad = -2.0

# de/db = 2.0  # value of a in a*b
# dL/db = de/db * dL/de = 2.0 * -2.0 = -4.0
b._grad = -4.0

# de/da = -3.0  # value of b in a*b
# dL/da = de/da * dL/de = -3.0 * -2.0 = 6.0
a._grad = 6.0

In [ ]:
build_dot_graph(L)

In [ ]:
def lol():
    """Numerically test gradients"""
    h = 0.00001

    a = Value(2.0 - h, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2, label='f')
    L = d * f; L.label='L'
    L1 = L.data

    a = Value(2.0 + h, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2, label='f')
    L = d * f; L.label='L'
    L2 = L.data

    derivative = (L2-L1) / (2*h)
    print(derivative)

# 6.000000000128124
lol()

In [ ]:
# Bump leaf nodes in direction of grad. L should increase
a.data += 0.01 * a._grad
b.data += 0.01 * b._grad
c.data += 0.01 * c._grad
f.data += 0.01 * f._grad

e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
L = d * f; L.label='L'

# Value(-7.286496)
print(L)